# Graph convolutional network classifier

Setting up the code environment

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Add pre-installed pytorch3d to sys.path
import sys
sys.path.append("/content/drive/My Drive/GoogleColab/pytorch3d_packages")

The classifier

In [ ]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

from pytorch3d.datasets import ShapeNetCore
from pytorch3d.structures import Meshes
from pytorch3d.ops import GraphConv


# Unzipping the dataset
# Note, problem with the code. For extract_path, it might need to run once and get an error,
# then change the variable to extract_path = "/content/ShapeNetCore/ShapeNetCore/ShapeNetCore"

zip_path = "/content/drive/My Drive/GoogleColab/ShapeNetCore.zip"
extract_path = "/content/ShapeNetCore/ShapeNetCore/"

if not os.path.exists(extract_path):
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(extract_path))
    print("Extraction complete.")
else:
    print("Dataset already extracted.")

print(os.listdir(extract_path))


# Variables
categories = ["03642806", "03211117", "03046257", "02992529", "02808440"]
category_names = ["Laptop", "Monitor", "Clock", "Cellphone", "Bathtub"]
categories_to_idx = {s: i for i, s in enumerate(categories)}
batch_size = 8
lr = 0.001
epochs = 5
device = torch.device("cpu") # GPU didnt work for this code

print("Using device:", device)


# Help functions to build the edges
def to_undirected(edges):
    rev = edges[:, [1, 0]]
    all_e = torch.cat([edges, rev], dim=0)
    return all_e.unique(dim=0)

def build_edges_from_faces(faces):
    edges = torch.cat([
        faces[:, [0, 1]],
        faces[:, [1, 2]],
        faces[:, [2, 0]]
    ], dim=0)
    return to_undirected(edges)


# Converts batch of meshes into batched graph data
def collate_shapenet(batch):
    verts_list = [b["verts"] for b in batch]
    faces_list = [b["faces"] for b in batch]
    synset_ids = [b["synset_id"] for b in batch]
    labels = torch.tensor([categories_to_idx[sid] for sid in synset_ids], dtype=torch.long)

    mesh_batch = Meshes(verts=verts_list, faces=faces_list)

    xs, edges_list, batch_vec = [], [], []
    offset = 0
    for i, (v, f) in enumerate(zip(mesh_batch.verts_list(), mesh_batch.faces_list())):
        edges = build_edges_from_faces(f)
        edges_list.append(edges + offset)
        xs.append(v)
        batch_vec.append(torch.full((v.size(0),), i, dtype=torch.long))
        offset += v.size(0)

    verts = torch.cat(xs, dim=0)
    edges = torch.cat(edges_list, dim=0)
    batch_vec = torch.cat(batch_vec)

    return verts, edges, batch_vec, labels


# The graph convolutional network
class GraphConvNetwork(nn.Module):
    def __init__(self, in_channels=3, hidden_channels=64, num_classes=5, num_layers=3):
        super().__init__()
        self.convs = nn.ModuleList()
        self.convs.append(GraphConv(in_channels, hidden_channels))
        for _ in range(max(0, num_layers - 2)):
            self.convs.append(GraphConv(hidden_channels, hidden_channels))
        self.convs.append(GraphConv(hidden_channels, hidden_channels))
        self.fc = nn.Linear(hidden_channels, num_classes)

    def forward(self, verts, edges, batch):
        x = verts
        for conv in self.convs:
            x = F.relu(conv(x, edges))

        # Global mean pooling
        num_graphs = int(batch.max().item() + 1)
        out = torch.zeros(num_graphs, x.size(1), device=x.device)
        out.index_add_(0, batch, x)
        counts = torch.bincount(batch)
        out = out / counts.unsqueeze(-1)
        return self.fc(out)

# Getting the dataset
dataset = ShapeNetCore(
    data_dir=extract_path,
    synsets=categories,
    version=2,
    load_textures=False
)
print("Total models loaded:", len(dataset))

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
                          collate_fn=collate_shapenet, num_workers=0)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_shapenet, num_workers=0)


# Training the model
model = GraphConvNetwork(in_channels=3, hidden_channels=64, num_classes=len(categories)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for verts, edges, batch_vec, labels in train_loader:
        verts, edges, batch_vec, labels = verts.to(device), edges.to(device), batch_vec.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(verts, edges, batch_vec)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    correct, total = 0, 0
    class_correct = [0]*len(categories)
    class_total = [0]*len(categories)
    with torch.no_grad():
        for verts, edges, batch_vec, labels in val_loader:
            verts, edges, batch_vec, labels = verts.to(device), edges.to(device), batch_vec.to(device), labels.to(device)
            outputs = model(verts, edges, batch_vec)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            for i in range(labels.size(0)):
                gt = labels[i].item()
                pred = predicted[i].item()
                class_total[gt] += 1
                if gt == pred:
                    class_correct[gt] += 1

    val_acc = correct / total if total > 0 else 0.0
    avg_loss = total_loss / (len(train_loader) if len(train_loader) > 0 else 1)
    print(f"Epoch {epoch+1}/{epochs}  Loss: {avg_loss:.4f}  Val Acc: {val_acc:.4f}")
    for i, name in enumerate(category_names):
        if class_total[i] > 0:
            print(f"  {name}: {class_correct[i]/class_total[i]:.4f}")
    print()


Using device: cpu
Dataset already extracted.
['03642806', '02808440', '03046257', '02992529', '03211117']
Total models loaded: 3891
Epoch 1/5  Loss: 1.0506  Val Acc: 0.5353
  Laptop: 0.0938
  Monitor: 0.7830
  Clock: 0.2366
  Cellphone: 0.3023
  Bathtub: 0.9464

Epoch 2/5  Loss: 0.7776  Val Acc: 0.7381
  Laptop: 0.8438
  Monitor: 0.8255
  Clock: 0.2061
  Cellphone: 0.8488
  Bathtub: 0.8690

Epoch 3/5  Loss: 0.7066  Val Acc: 0.7664
  Laptop: 0.8229
  Monitor: 0.8113
  Clock: 0.4427
  Cellphone: 0.8198
  Bathtub: 0.8750

Epoch 4/5  Loss: 0.6752  Val Acc: 0.7599
  Laptop: 0.8750
  Monitor: 0.8915
  Clock: 0.1756
  Cellphone: 0.8895
  Bathtub: 0.8512

Epoch 5/5  Loss: 0.6288  Val Acc: 0.7766
  Laptop: 0.8646
  Monitor: 0.8255
  Clock: 0.3053
  Cellphone: 0.8953
  Bathtub: 0.9107

